The necessary packages are imported.

In [ ]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

The dataset is read into ‘df’ dataframe.

In [ ]:
# Reading the file
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

Let us understand the dataframe ‘df’.

In [ ]:
df.shape # Shape of ‘df’

In [ ]:
df.columns # Prints columns of ‘df’


The columns are [‘Pregnancies’, ‘Glucose’, ‘BloodPressure’, ‘SkinThickness’, ‘Insulin’, ‘BMI’, ‘DiabetesPedigreeFunction’, ‘Age’, ‘Outcome’]


In [ ]:
df.describe() # Displays properties of each column

All the columns have count = 768 which suggests there are no missing values. The mean of ‘Outcome’ is 0.35 which suggests there are more ‘Outcome’ = 0 than ‘Outcome’ = 1 cases in the given dataset.
The dataframe ‘df’ is converted into numpy array ‘dataset’

In [ ]:
dataset = df.values

The ‘dataset’ is split into input X and output y


In [ ]:
X = dataset[:,0:8]
y = dataset[:,8].astype("int")

## Standardization
It can be observed that mean value of columns are very different. Hence the dataset is to be standardized so that no inappropriate weightage is given to any feature.

In [ ]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

Now let us look aat mean and standard deviation of ‘X_standardized’.


In [ ]:
pd.DataFrame(X_standardized).describe()

Mean of all columns is around 0 and Standard deviation of all columns is around 1. The data has been standardized.

## Tuning of Hyperparameters :- Batch Size and Epochs


In [ ]:
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

The neural architecture and optimization algorithm are defined. The neural network consists of 1 input layer, 2 hidden layers with rectified linear unit activation function and 1 output layer with sigmoid activation function. Adams optimization is chosen as the optimization algorithm for the neural network model.

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation="relu"))
    model.add(Dense(4, input_dim=8, kernel_initializer='normal', activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    
    adam = Adam(lr=0.01)
    model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])
    return model

We run the grid search for 2 hyperparameters :- ‘batch_size’ and ‘epochs’. The cross validation technique used is K-Fold with the default value k = 3. The accuracy score is calculated.

In [ ]:
# Create the model
model = KerasClassifier(build_fn = create_model,verbose = 0)
# Define the grid search parameters
batch_size = [10,20,40]
epochs = [10,50,100]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

The results are summarized. The best accuracy score and the best values of hyperparameters are printed.

In [ ]:
# Summarize the results
print("Best : {}, using {}".format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
for mean, stdev, param in zip(means, stds, params):
  print("{},{} with: {}".format(mean, stdev, param))

The best accuracy score is 0.7604 for ‘batch_size’ = 40 and ‘epochs’ = 10. So we choose ‘batch_size’ = 40 and ‘epochs’ = 10 while tuning other hyperparameters.

## Tuning of Hyperparameters:- Learning rate and Drop out rate
The learning rate plays an important role in optimization algorithm. If the learning rate is too large, the algorithm may diverge and thus can’t find the local optima. If the learning rate is too small, the algorithm may take many iterations to converge which results in high computational power and time. Thus we need an optimum value of learning rate which is small enough for the algorithm to converge and large enough to fasten the converging process. The learning rate helps with ‘Early Stopping’ which is a regularization method where the training set is trained as long as test set accuracy is increasing.

Drop out is a regularization method that reduces the complexity of the model and thus prevents overfitting the training data. By dropping an activation unit out, we mean temporarily removing it from the network, along with all its incoming and outgoing connections. Dropout rate can take values between 0 and 1. 0 implies no activation units are knocked out and 1 implies all the activation units are knocked out.

In [ ]:
from keras.layers import Dropout

# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

# Summarize the results

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}'.format(mean, stdev, param))

The best accuracy score is 0.7695 for ‘dropout_rate’ = 0.1 and ‘learning_rate’ = 0.001. So we choose ‘dropout_rate’ = 0.1 and ‘learning_rate’ = 0.001 while tuning other hyperparameters.

## Tuning of Hyperparameters:- Activation Function and Kernel Initializer
Activation functions introduce non-linear properties to the neural network such that non-linear complex functional mappings between input and output can be established. If we do not apply the activation function, then the output would be a simple linear function of the input.

The neural network needs to start with some weights and then iteratively update them to better values. Kernel initializer decides the statistical distribution or function to be used for initializing the weights.

In [ ]:
# Defining the model

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters

param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

# Summarize the results

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}'.format(mean, stdev, param))

The best accuracy score is 0.7591 for ‘activation_function’ = tanh and ‘kernel_initializer’ = uniform. So we choose ‘activation_function’ = tanh and ‘kernel_initializer’ = uniform while tuning other hyperparameters.

## Tuning of Hyperparameter :-Number of Neurons in activation layer
The complexity of the data has to be matched with the complexity of the model. The number of neurons in activation layer decides the complexity of the model. Higher the number of neurons in activation layer, higher is the degree of non-linear complex functional mappings between input and output.


In [ ]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

# Summarize the results

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}'.format(mean, stdev, param))

The best accuracy score is 0.7591 for number of neurons in first layer = 16 and number of neurons in second layer = 4.

## Training model with optimum values of Hyperparameters
The model is trained using optimum values of hyperparameters found in previous section.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 8,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 16,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Fitting the model

model.fit(X_standardized,y)

# Predicting using trained model

y_predict = model.predict(X_standardized)

# Printing the metrics

print(accuracy_score(y,y_predict))
print(classification_report(y,y_predict))


We get an accuracy of 77.6% and F1 scores of 0.84 and 0.65.
The hyperparameter optimization was carried out by taking 2 hyperparameters at once. We may have missed the best values. The performance can be further improved by finding the optimum values of hyperparameters all at once given by the code snippet below. Note:- This process is computationally expensive.

In [ ]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0)

# Define the grid search parameters

batch_size = [10,20,40]
epochs = [10,50,100]
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']
neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

# Summarize the results

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}'.format(mean, stdev, param))